## Importerer pakker

In [14]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

## Leser inn data

In [15]:
demographic_df = pd.read_csv('../raw_data/demographic.csv')
hospital_df = pd.read_csv('../raw_data/hospital.csv')
physiological_df = pd.read_table('../raw_data/physiological.txt')
severity_df = pd.read_json('../raw_data/severity.json')


## Deler demographic_df i aldersgrupper

In [16]:
# Deler opp alder
bins = [-10, 18, 40, 60, 80, 120]
labels = ['ugyldige data', '18-39','40-59','60-79','80+']
demographic_df['aldersgrupper'] = pd.cut(demographic_df['alder'], bins=bins, right=False, labels=labels)
demographic_df.loc[demographic_df['alder'] < 0]


,pasient_id,alder,kjønn,utdanning,inntekt,etnisitet,aldersgrupper
13,15,-1.0,female,NaN,NaN,white,ugyldige data
93,112,-1.0,female,12.0,under $11k,white,ugyldige data
219,259,-1.0,male,13.0,NaN,NaN,ugyldige data
359,421,-1.0,male,NaN,NaN,black,ugyldige data
868,1002,-1.0,male,12.0,$25-$50k,black,ugyldige data


## Fjerner duplikat pasient_ider

In [17]:
# Pasient_id 4 og 59 er doble like variabler
# Derfor kan vi slette en av de 
#demographic_df[demographic_df['pasient_id']==4]
#demographic_df[demographic_df['pasient_id']==59]
demographic_df = demographic_df.drop([7740, 7741])
demographic_df



,pasient_id,alder,kjønn,utdanning,inntekt,etnisitet,aldersgrupper
0,2,60.33899,female,12.0,$11-$25k,white,60-79
1,3,52.74698,female,12.0,under $11k,white,40-59
2,4,42.38498,female,11.0,under $11k,white,40-59
3,5,79.88495,female,NaN,NaN,white,60-79
4,6,93.01599,male,14.0,NaN,white,80+
...,...,...,...,...,...,...,...
7735,9101,66.07300,male,8.0,NaN,white,60-79
7736,9102,55.15399,female,11.0,NaN,white,40-59
7737,9103,70.38196,male,NaN,NaN,white,60-79
7738,9104,47.01999,male,13.0,NaN,white,40-59


## Deler demographic_df i utdanningsnivåer

In [18]:
bins = [0, 10 , 13, 60]
labels = ['lav utdanning', 'middels utdanning','høy utdanning']
demographic_df['utdanningsnivå'] = pd.cut(demographic_df['utdanning'], bins=bins, right=False, labels=labels)

## Forbereder Hospital_df

In [19]:
hospital_df

,pasient_id,sykehusdød,oppholdslengde
0,2,1,4
1,3,0,17
2,4,0,3
3,5,0,-99
4,6,1,4
...,...,...,...
7735,9101,0,23
7736,9102,0,29
7737,9103,0,8
7738,9104,1,7


## Fjerner pasienter med negativ oppholdslengde

In [20]:
negative_oppholdslengde = hospital_df[hospital_df['oppholdslengde'] < 0]
negative_oppholdslengde_count = negative_oppholdslengde['oppholdslengde'].value_counts()
hospital_df[hospital_df['oppholdslengde']==-99]

# Fjerner pasient id 5, 23, 102, 256, 508, 8000 for å rense data
pasient_ids_to_remove = [5, 23, 102, 256, 508, 8000]
index_to_drop = hospital_df[hospital_df['pasient_id'].isin(pasient_ids_to_remove)].index
hospital_df = hospital_df.drop(index_to_drop)
hospital_df

,pasient_id,sykehusdød,oppholdslengde
0,2,1,4
1,3,0,17
2,4,0,3
4,6,1,4
5,7,0,9
...,...,...,...
7735,9101,0,23
7736,9102,0,29
7737,9103,0,8
7738,9104,1,7


## Fjerner pasient_idene som er ugyldige i alle datasett

In [21]:
pasient_ids_to_remove = [5, 23, 102, 256, 508, 8000]
index_to_drop = demographic_df[demographic_df['pasient_id'].isin(pasient_ids_to_remove)].index
demographic_df = demographic_df.drop(index_to_drop)
demographic_df

,pasient_id,alder,kjønn,utdanning,inntekt,etnisitet,aldersgrupper,utdanningsnivå
0,2,60.33899,female,12.0,$11-$25k,white,60-79,middels utdanning
1,3,52.74698,female,12.0,under $11k,white,40-59,middels utdanning
2,4,42.38498,female,11.0,under $11k,white,40-59,middels utdanning
4,6,93.01599,male,14.0,NaN,white,80+,høy utdanning
5,7,62.37097,male,14.0,$25-$50k,white,60-79,høy utdanning
...,...,...,...,...,...,...,...,...
7735,9101,66.07300,male,8.0,NaN,white,60-79,lav utdanning
7736,9102,55.15399,female,11.0,NaN,white,40-59,middels utdanning
7737,9103,70.38196,male,NaN,NaN,white,60-79,NaN
7738,9104,47.01999,male,13.0,NaN,white,40-59,høy utdanning


In [22]:
pasient_ids_to_remove = [5, 23, 102, 256, 508, 8000]
index_to_drop = physiological_df[physiological_df['pasient_id'].isin(pasient_ids_to_remove)].index
physiological_df = physiological_df.drop(index_to_drop)
physiological_df

,pasient_id,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde
0,2,43.0,17.097656,112.0,34.0,34.59375,98.00000,NaN,NaN,5.500000,132.0,7.250000,NaN,NaN,NaN
1,3,70.0,8.500000,88.0,28.0,37.39844,231.65625,NaN,2.199707,2.000000,134.0,7.459961,NaN,NaN,NaN
2,4,75.0,9.099609,88.0,32.0,35.00000,NaN,NaN,NaN,0.799927,139.0,NaN,NaN,NaN,NaN
4,6,110.0,10.398438,101.0,44.0,38.39844,266.62500,NaN,NaN,0.699951,140.0,7.659180,NaN,NaN,NaN
5,7,78.0,11.699219,120.0,28.0,37.39844,309.50000,4.799805,0.399963,1.599854,132.0,7.479492,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7735,9101,109.0,7.399414,104.0,22.0,35.69531,280.00000,3.699707,0.399963,1.099854,131.0,7.459961,188.0,21.0,NaN
7736,9102,43.0,NaN,0.0,8.0,38.59375,218.50000,NaN,NaN,5.899414,135.0,7.289062,190.0,49.0,0.0
7737,9103,111.0,8.398438,83.0,24.0,36.69531,180.00000,NaN,0.399963,2.699707,139.0,7.379883,189.0,60.0,3900.0
7738,9104,99.0,7.599609,110.0,24.0,36.39844,428.56250,1.199951,0.399963,3.500000,135.0,7.469727,246.0,55.0,NaN


In [ ]:
pasient_ids_to_remove = [5, 23, 102, 256, 508, 8000]

## Forbereder Physioligical_df

In [23]:
severity_df

,sykdomskategori_id,sykdomskategori,pasient_id,dødsfall,sykdom_underkategori,antall_komorbiditeter,koma_score,adl_pasient,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,dnr_dag
0,A1s,ARF/MOSF,"[5, 15, 18, 23, 28, 34, 39, 43, 46, 47, 48, 58...","[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, ...","[ARF/MOSF w/Sepsis, ARF/MOSF w/Sepsis, ARF/MOS...","[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 0, 3, 1, 3, ...","[26.0, 26.0, 26.0, 0.0, 26.0, 37.0, 0.0, 0.0, ...","[None, None, None, None, None, None, None, Non...","[2.0, None, 0.0, 5.0, 2.0, None, 0.0, None, No...","[23.5, 30.5, 40.296875, 31.6992188, 46.796875,...","[30.0, 39.0, 58.0, 42.0, 85.0, 49.0, 5.0, 76.0...","[0.6348876950000001, 0.590942383, 0.2129821780...","[0.5329589840000001, 0.481994629, 0.1169891360...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, yes, no, no, no, no, no, yes,...","[0.899999619, 0.899999619, 0.09999996400000001...","[0.9, 0.9, 0.001, 0.5, 0.000125, 0.60000000000...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."
1,BrY,COPD/CHF/Cirrhosis,"[2, 3, 7, 8, 11, 13, 14, 19, 20, 30, 31, 32, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[Cirrhosis, Cirrhosis, CHF, CHF, CHF, Cirrhosi...","[2, 2, 1, 3, 1, 1, 0, 2, 1, 2, 2, 2, 2, 1, 1, ...","[44.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[None, 1.0, 0.0, None, 2.0, 0.0, 0.0, 7.0, 3.0...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 7.0, None,...","[52.6953125, 20.5, 17.296875, 21.5976562, 14.5...","[74.0, 45.0, 46.0, 53.0, 14.0, 30.0, 34.0, 42....","[0.000999928, 0.790893555, 0.892944336, 0.6708...","[0.0, 0.6649169920000001, 0.820922852, 0.49896...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, no, no, no, no, no, no, no, n...","[0.0, 0.75, None, 0.799999714, 0.699999809, No...","[0.0, 0.5, 0.7000000000000001, 0.4, 0.5, None,...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."
2,ChE,Cancer,"[4, 9, 10, 12, 16, 17, 21, 24, 27, 41, 42, 54,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[Lung Cancer, Lung Cancer, Colon Cancer, Lung ...","[2, 2, 0, 0, 1, 2, 0, 0, 0, 1, 1, 1, 2, 0, 1, ...","[0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 44.0...","[0.0, None, 0.0, 1.0, 2.0, None, 0.0, None, No...","[0.0, 7.0, None, 1.0, 0.0, None, 0.0, None, No...","[20.0976562, 15.8984375, 2.2998047, 16.3984375...","[19.0, 17.0, 9.0, 17.0, 11.0, 4.0, 16.0, 11.0,...","[0.6989746090000001, 0.570922852, 0.9528808590...","[0.411987305, 0.24899292, 0.8879394530000001, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[metastatic, metastatic, metastatic, metastati...","[0.899999619, 0.049999982000000005, None, None...","[0.5, 0.000125, None, None, 0.7000000000000001...","[None, dnr ved innleggelse, None, None, None, ...","[None, 0.0, None, None, None, None, None, None..."
3,DWw,Coma,"[6, 162, 188, 250, 252, 262, 275, 309, 323, 35...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...","[Coma, Coma, Coma, Coma, Coma, Coma, Coma, Com...","[1, 0, 0, 2, 1, 2, 1, 0, 0, 1, 1, 0, 2, 2, 0, ...","[55.0, 61.0, 94.0, 55.0, 94.0, 100.0, 100.0, 5...","[None, None, None, None, None, None, None, Non...","[1.0, None, None, 1.0, None, None, None, None,...","[19.3984375, 30.3984375, 20.296875, 30.8984375...","[27.0, 36.0, 22.0, 53.0, 40.0, 25.0, 58.0, 16....","[0.28497314500000004, 0.438964844, 0.280944824...","[0.214996338, 0.365966797, 0.211975098, 0.2729...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, yes, no, no, no, no, no, no, ...","[0.0, None, None, None, None, 0.09999996400000...","[0.0, None, N

In [24]:
physiological_df['pasient_id'].value_counts()

2       1
6111    1
6074    1
6073    1
6072    1
       ..
3013    1
3012    1
3011    1
3010    1
9105    1
Name: pasient_id, Length: 7734, dtype: int64

## Sammenligner pasient_id i datasettene

In [25]:
# Uthente pasient_id kolonner
demographic_ids = demographic_df.iloc[:, 0]
physiological_ids = physiological_df.iloc[:, 0]
hospital_ids = hospital_df.iloc[:, 0]
severity_ids = severity_df.iloc[:, 3]

# Finner forskjeller
ids_in_demo_not_in_physio = set(demographic_ids) - set(physiological_ids)
ids_in_physio_not_in_demo = set(physiological_ids) - set(demographic_ids)
ids_in_hospital_not_in_demo = set(hospital_ids) - set(demographic_ids)

# Se om det er noen forskjeller
if not ids_in_demo_not_in_physio and not ids_in_physio_not_in_demo and not ids_in_hospital_not_in_demo:
    print("No mismatched IDs, all DataFrames have the same pasient_id values.")
else:
    print("There are mismatched IDs.")

No mismatched IDs, all DataFrames have the same pasient_id values.


In [26]:
flattened_records = []

# Iterate through each entry in json_data
for entry in severity_df:
    # Zip together the list values by pasient_id
    for i in range(len(entry['pasient_id'])):
        flattened_record = {
            "pasient_id": entry['pasient_id'][i],
            "dødsfall": entry['dødsfall'][i],
            "sykdom_underkategori": entry['sykdom_underkategori'][i],
            "antall_komorbiditeter": entry['antall_komorbiditeter'][i],
            "koma_score": entry['koma_score'][i],
            "adl_pasient": entry['adl_pasient'][i],
            "adl_stedfortreder": entry['adl_stedfortreder'][i],
            "fysiologisk_score": entry['fysiologisk_score'][i],
            "apache_fysiologisk_score": entry['apache_fysiologisk_score'][i],
            "sykdomskategori_id": entry['sykdomskategori_id'],
            "sykdomskategori": entry['sykdomskategori']
        }
        # Append each flattened record to the list
        flattened_records.append(flattened_record)

df = pd.DataFrame(flattened_records)
df


TypeError: string indices must be integers